In [ ]:
import keras
from deepweeds.models import not_resnet
import tensorflow as tf
import onnxruntime as rt
from pathlib import Path
import tf2onnx
from omegaconf import OmegaConf

In [ ]:
cfg = OmegaConf.load('../conf/config.yaml')
model_path = Path('../outputs/2025-02-20/14-04-37/MeNet.keras')
keras.mixed_precision.set_global_policy('float32')
model = not_resnet(**cfg.model)
model.load_weights(model_path)

In [ ]:
# model = keras.saving.load_model(model_path)

In [ ]:
dtypes = []
for layer in model.layers:
    for w in layer.get_weights():
        dtypes.append(w.dtype)
    # weights = [w.astype('float32') for w in weights]
    # layer.set_weights(weights)

assert all(w == 'float32' for w in dtypes)

In [ ]:
onnx_path = Path('./models') / f"{model.name}.onnx"
spec = (tf.TensorSpec((None, 256, 256, 3), tf.float32, name="input"),)
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=onnx_path)
output_names = [n.name for n in model_proto.graph.output]
# model.export(onnx_path, format='onnx', input_signature=input_spec)

In [ ]:
del model

In [ ]:
m = rt.InferenceSession(onnx_path)